## Functions

In [12]:

import LyndonWords
import importlib
importlib.reload(LyndonWords)
from LyndonWords import *
import itertools
from tqdm.notebook import tqdm
import math
import numpy as np
def genPermutations(type,n,k):
    rootSystemPermutations = np.empty(math.factorial(n+1),dtype=object)
    v = 0
    for i in tqdm(itertools.permutations(np.array(range(n+1))),total=math.factorial(n+1)):
        rootSystemPermutations[v] = rootSystem(i,type,k)
        v+=1
    return rootSystemPermutations

In [13]:
def checkMonotoneIncreasingImpDeltaStart(rootSystemPermutations):
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.getWords(rootsys.delta)
        temp = rootsys.checkMonotonicity("Increasing")
        monotoneIncreasing = [rootsys.getWords(i[0] + rootsys.delta)[0] for i in temp]
        deltaArrays = [i.string for i in deltaWords]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(str(i).startswith(str(j))):
                    flag = True
                    break
            if(flag):
                continue
            for j in rootsys.getAffineWords(i.weights):
                if np.any(np.all(j[0:sum(rootsys.delta)] == deltaArrays)):
                    returnArr.append((rootsys.ordering,j))
    return returnArr

In [14]:
def checkDeltaStartImpMonotoneIncreasing(rootSystemPermutations):
    returnArr = []
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.getWords(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.getWords(base + rootsys.delta)[0]
            flag = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaHeight] == i)):
                    flag = True
                    break
            if not flag:
                continue
            if(rootsys.getMonotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.getWords(base)[0])))
    return returnArr

In [15]:
def printByLength(rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.getWords(comb):
                print(" "+str(i))

In [16]:
def printByBaseWord(rootSystem:rootSystem,comb):
    k=0
    isImaginary = rootSystem.isImaginary(comb)
    for i in rootSystem.getAffineWords(comb):
        if(isImaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not isImaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [17]:
def exportByLength(rootSystem,quoteMarks=False):
    file = open("out.txt",'w')
    file.write(str(rootSystem.ordering)+"\n")
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            for i in rootSystem.getWords(comb):
                if(quoteMarks):
                    file.write(f"'{str(i)}',")
                else:
                    file.write(str(i) + '\n')
    file.close()

In [18]:
def checkDeltaPattern(perms,k=2,printDelta = False):
    for i in perms:
        baseImaginaryWords = i.getWords(i.delta)
        deltaWords = i.getWords(k*i.delta)
        flag = True
        for j in range(1,len(deltaWords)):
            if(deltaWords[j][i.deltaHeight-1] != deltaWords[j][0]):
                if(flag):
                    print(i.ordering)
                    flag = False
                if(printDelta):
                    print(f"  SL_{j}(\delta): " + '-'.join([str(i) for i in i.costfac(i.getWords(i.delta)[j])]))
                print(f"  SL_{j}({k}*\delta): " + '-'.join([str(i) for i in i.costfac((deltaWords[j]))]))

In [19]:
def checkHDeltasRepeat(perms):
    excepts = []
    for i in perms:
        hs = [j.hs for j in i.getAffineWords(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

In [20]:
def compareRootSystems(r1,r2):
    words = list(r1.weightToWordDictionary.values())
    for i in words:
        for j in range(len(i)):
            if(str(i[j]) != str(r2.getWords(i[j].weights)[j])):
                yield (i[j].weights,str(i[j]),str(r2.getWords(i[j].weights)[j]))

In [21]:
def generateEquivalence(permutations):
    equivalences = np.arange(len(permutations))
    for i in range(len(permutations)):
        for j in range(0,i):
            if(equivalences[j] != j):
                continue
            flag = False
            for k in compareRootSystems(permutations[i],permutations[j]):
                flag =True
                break
            if(not flag):
                yield (permutations[i].ordering,permutations[j].ordering)

In [22]:
exportByLength(rootSystem([1,4,5,2,3,0],'D',3),True)

## Type C

In [23]:
Cperms = genPermutations('C',5,3)

  0%|          | 0/720 [00:00<?, ?it/s]

In [24]:
exceptions = checkMonotoneIncreasingImpDeltaStart(Cperms)
if(len(exceptions) != 0):
    print("Exception")

Above shows that monotone increasing implies it starts with a delta word

In [25]:
checkDeltaStartImpMonotoneIncreasing(Cperms)

[('0<1<2<3<4<5', '1'),
 ('0<1<2<3<4<5', '2'),
 ('0<1<2<3<4<5', '3'),
 ('0<1<2<3<4<5', '4'),
 ('0<1<2<3<4<5', '5'),
 ('0<1<2<3<4<5', '4,5'),
 ('0<1<2<3<4<5', '1,2'),
 ('0<1<2<3<4<5', '2,3'),
 ('0<1<2<3<4<5', '3,4'),
 ('0<1<2<3<4<5', '3,4,5'),
 ('0<1<2<3<4<5', '1,2,3'),
 ('0<1<2<3<4<5', '2,3,4'),
 ('0<1<2<3<4<5', '4,4,5'),
 ('0<1<2<3<4<5', '2,3,4,5'),
 ('0<1<2<3<4<5', '3,4,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4'),
 ('0<1<2<3<4<5', '1,2,3,4,5'),
 ('0<1<2<3<4<5', '2,3,4,5,4'),
 ('0<1<2<3<4<5', '3,4,3,4,5'),
 ('0<1<2<3<4<5', '1,2,3,4,5,4'),
 ('0<1<2<3<4<5', '2,3,4,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,4,5,4,3'),
 ('0<1<2<3<4<5', '2,3,4,2,3,4,5'),
 ('0<1<2<3<4<5', '1,2,3,4,5,4,3,2'),
 ('0<1<2<3<4<5', '1,2,3,4,1,2,3,4,5'),
 ('0<1<2<3<5<4', '1'),
 ('0<1<2<3<5<4', '2'),
 ('0<1<2<3<5<4', '3'),
 ('0<1<2<3<5<4', '4'),
 ('0<1<2<3<5<4', '5'),
 ('0<1<2<3<5<4', '5,4'),
 ('0<1<2<3<5<4', '1,2'),
 ('0<1<2<3<5<4', '2,3'),
 ('0<1<2<3<5<4', '3,4'),
 ('0<1<2<3<5<4', '3,4,5'),
 ('0<1<2<3<5<4', '1,2,3'),
 ('0<1<2<3<5<4',

In [26]:
C205134 = rootSystem([2,0,5,1,3,4],'C',5)
for i in C205134.getAffineWords([0,0,1,0,0,0]):
    print(*C205134.costfac(i),sep='-')


3-None
2,3,1,0,1-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,0,2,3,4,1,5-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5,2,3,4,1,0,2,3,4,1,5-2,3,4,5,4,3


Implies that beginning with a k-delta word implies monotone increasing

In [42]:
checkDeltaPattern(Cperms,2,printDelta=True)

2<0<3<1<4<5
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(2*\delta): 2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1
2<0<3<1<5<4
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(2*\delta): 2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1
2<0<3<4<1<5
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(2*\delta): 2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1
2<0<3<4<5<1
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(2*\delta): 2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1
2<0<3<5<1<4
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(2*\delta): 2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1
2<0<3<5<4<1
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(2*\delta): 2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1
2<0<4<1<3<5
  SL_3(\delta): 2,3,4,5,4-2,3,1,0,1
  SL_3(2*\delta): 2,3,4,5,4,2,3,1,4,0,2,3,1,4,5-2,3,1,0,1
2<0<4<1<5<3
  SL_3(\delta): 2,3,4,5,4-2,3,1,0,1
  SL_3(2*\delta): 2,3,4,5,4,2,3,1,4,0,2,3,1,4,5-2,3,1,0,1
2<0<4<3<1<5
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,1
  SL_4(2*\delta): 2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1
2<0<4<3<5<1
  SL_4(\delta): 2,3,4,5,4,3-2,1,0,

In [28]:
checkHDeltasRepeat(Cperms)

[]

In [29]:
index=250
print(Cperms[index].ordering)
[print(*list(Cperms[index].costfac(i))+[i.hs],sep='-',) for i in Cperms[index].getAffineWords(Cperms[index].delta)]

2<0<3<5<1<4
2,1,3,4,0-2,1,3,4,5-[1 1 1 1 1]
2,1,3,4-2,1,3,4,5,0-[1 1 1 1 0]
2,1,3,0-2,1,3,4,5,4-[1 1 1 2 1]
2,1,0-2,1,3,4,5,4,3-[1 1 2 2 1]
2,3,4,5,4,3-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [30]:
for i in generateEquivalence(Cperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<0<4<5 

## Type G

In [32]:
G2perms = genPermutations('G',2,5)

  0%|          | 0/6 [00:00<?, ?it/s]

In [33]:
checkDeltaStartImpMonotoneIncreasing(G2perms)

[('0<1<2', '1'),
 ('0<1<2', '2'),
 ('0<1<2', '1,2'),
 ('0<1<2', '1,2,2'),
 ('0<1<2', '1,2,2,2'),
 ('0<1<2', '1,2,1,2,2'),
 ('0<2<1', '1'),
 ('0<2<1', '2'),
 ('0<2<1', '2,1'),
 ('0<2<1', '2,2,1'),
 ('0<2<1', '2,2,2,1'),
 ('0<2<1', '2,2,1,2,1'),
 ('1<0<2', '0'),
 ('1<0<2', '1,2,2'),
 ('1<0<2', '1,2,2,0'),
 ('1<2<0', '2'),
 ('1<2<0', '1,0'),
 ('1<2<0', '1,0,2'),
 ('1<2<0', '1,0,2,2'),
 ('1<2<0', '1,0,2,2,2'),
 ('2<0<1', '0'),
 ('2<0<1', '2,1'),
 ('2<0<1', '2,1,0'),
 ('2<0<1', '2,1,2,1,0'),
 ('2<1<0', '0'),
 ('2<1<0', '2,1'),
 ('2<1<0', '2,1,0'),
 ('2<1<0', '2,1,2,1,0')]

In [34]:
checkMonotoneIncreasingImpDeltaStart(G2perms)

[]

In [35]:
for i in G2perms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [40]:
checkDeltaPattern(G2perms)

In [41]:
checkHDeltasRepeat(G2perms)

[]

In [39]:
for i in generateEquivalence(G2perms):
    print(f"{i[0]} = {i[1]}")

## Type B

In [43]:
Bperms = genPermutations('B',5,4)

  0%|          | 0/720 [00:00<?, ?it/s]

In [44]:
checkDeltaStartImpMonotoneIncreasing(Bperms)

[('0<1<2<3<4<5', '1'),
 ('0<1<2<3<4<5', '2'),
 ('0<1<2<3<4<5', '3'),
 ('0<1<2<3<4<5', '4'),
 ('0<1<2<3<4<5', '5'),
 ('0<1<2<3<4<5', '4,5'),
 ('0<1<2<3<4<5', '1,2'),
 ('0<1<2<3<4<5', '2,3'),
 ('0<1<2<3<4<5', '3,4'),
 ('0<1<2<3<4<5', '3,4,5'),
 ('0<1<2<3<4<5', '1,2,3'),
 ('0<1<2<3<4<5', '2,3,4'),
 ('0<1<2<3<4<5', '4,5,5'),
 ('0<1<2<3<4<5', '2,3,4,5'),
 ('0<1<2<3<4<5', '1,2,3,4'),
 ('0<1<2<3<4<5', '3,4,5,5'),
 ('0<1<2<3<4<5', '1,2,3,4,5'),
 ('0<1<2<3<4<5', '2,3,4,5,5'),
 ('0<1<2<3<4<5', '3,4,5,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5'),
 ('0<1<2<3<4<5', '2,3,4,5,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5,4'),
 ('0<1<2<3<4<5', '2,3,4,5,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5,4,3,2'),
 ('0<1<2<3<5<4', '1'),
 ('0<1<2<3<5<4', '2'),
 ('0<1<2<3<5<4', '3'),
 ('0<1<2<3<5<4', '4'),
 ('0<1<2<3<5<4', '5'),
 ('0<1<2<3<5<4', '5,4'),
 ('0<1<2<3<5<4', '1,2'),
 ('0<1<2<3<5<4', '2,3'),
 ('0<1<2<3<5<4', '3,4'),
 ('0<1<2<3<5<4', '3,4,5'),
 ('0<1<2<3<5<4', '1,2,3'),
 ('0<1<2<3<5<4',

In [45]:
checkMonotoneIncreasingImpDeltaStart(Bperms)

[]

In [46]:
for i in Bperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [47]:
checkDeltaPattern(Bperms,printDelta=True)

2<0<4<1<3<5
  SL_4(\delta): 2,1,0-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,4,5,5,4,0,2,3,1,2,3,4,5,5,4,3
2<0<4<1<5<3
  SL_4(\delta): 2,1,0-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,4,5,5,4,0,2,3,1,2,3,4,5,5,4,3
2<0<4<5<1<3
  SL_4(\delta): 2,1,0-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,4,5,5,4,0,2,3,1,2,3,4,5,5,4,3
2<0<5<1<3<4
  SL_3(\delta): 2,3,1,0-2,3,4,5,5,4
  SL_3(2*\delta): 2,3,1,0-2,3,4,5,5,0,2,3,4,1,2,3,4,5,5,4
2<0<5<1<4<3
  SL_3(\delta): 2,3,1,0-2,3,4,5,5,4
  SL_3(2*\delta): 2,3,1,0-2,3,4,5,5,0,2,3,4,1,2,3,4,5,5,4
2<0<5<4<1<3
  SL_4(\delta): 2,1,0-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,4,5,5,4,0,2,3,1,2,3,4,5,5,4,3
2<1<4<0<3<5
  SL_4(\delta): 2,0,1-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,0,1-2,3,4,5,5,4,1,2,3,0,2,3,4,5,5,4,3
2<1<4<0<5<3
  SL_4(\delta): 2,0,1-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,0,1-2,3,4,5,5,4,1,2,3,0,2,3,4,5,5,4,3
2<1<4<5<0<3
  SL_4(\delta): 2,0,1-2,3,4,5,5,4,3
  SL_4(2*\delta): 2,0,1-2,3,4,5,5,4,1,2,3,0,2,3,4,5,5,4,3
2<1<5<0<3<4
  SL_3(\delta): 2,3,0,1-2,3,4,5,5,

In [48]:
checkHDeltasRepeat(Bperms)

[]

In [49]:
for i in generateEquivalence(Bperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<4<0<5 

## Type D

In [ ]:
Dperms = genPermutations('D',5,4)

  0%|          | 0/720 [00:00<?, ?it/s]

In [ ]:
checkMonotoneIncreasingImpDeltaStart(Dperms)

[]

In [ ]:
checkDeltaStartImpMonotoneIncreasing(Dperms)

[('0<1<2<3<4<5', '1'),
 ('0<1<2<3<4<5', '2'),
 ('0<1<2<3<4<5', '3'),
 ('0<1<2<3<4<5', '4'),
 ('0<1<2<3<4<5', '5'),
 ('0<1<2<3<4<5', '1,2'),
 ('0<1<2<3<4<5', '2,3'),
 ('0<1<2<3<4<5', '3,4'),
 ('0<1<2<3<4<5', '3,5'),
 ('0<1<2<3<4<5', '1,2,3'),
 ('0<1<2<3<4<5', '2,3,4'),
 ('0<1<2<3<4<5', '2,3,5'),
 ('0<1<2<3<4<5', '3,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4'),
 ('0<1<2<3<4<5', '1,2,3,5'),
 ('0<1<2<3<4<5', '2,3,5,4'),
 ('0<1<2<3<4<5', '1,2,3,5,4'),
 ('0<1<2<3<4<5', '2,3,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,5,4,3,2'),
 ('0<1<2<3<5<4', '1'),
 ('0<1<2<3<5<4', '2'),
 ('0<1<2<3<5<4', '3'),
 ('0<1<2<3<5<4', '4'),
 ('0<1<2<3<5<4', '5'),
 ('0<1<2<3<5<4', '1,2'),
 ('0<1<2<3<5<4', '2,3'),
 ('0<1<2<3<5<4', '3,4'),
 ('0<1<2<3<5<4', '3,5'),
 ('0<1<2<3<5<4', '1,2,3'),
 ('0<1<2<3<5<4', '2,3,4'),
 ('0<1<2<3<5<4', '2,3,5'),
 ('0<1<2<3<5<4', '3,4,5'),
 ('0<1<2<3<5<4', '1,2,3,4'),
 ('0<1<2<3<5<4', '1,2,3,5'),
 ('0<1<2<3<5<4', '2,3,4,5'),
 ('0<1<2<3<5<4', '1,2,3,4,5'),
 ('0<1<2<3<5<4', '2

In [ ]:
checkDeltaPattern(Dperms,2,printDelta=True)

2<0<4<5<1<3
  SL_4(\delta): 2,1,0-2,3,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,5,4,0,2,3,1,2,3,5,4,3
2<0<5<4<1<3
  SL_4(\delta): 2,1,0-2,3,4,5,3
  SL_4(2*\delta): 2,1,0-2,3,4,5,0,2,3,1,2,3,4,5,3
2<1<4<5<0<3
  SL_4(\delta): 2,0,1-2,3,5,4,3
  SL_4(2*\delta): 2,0,1-2,3,5,4,1,2,3,0,2,3,5,4,3
2<1<5<4<0<3
  SL_4(\delta): 2,0,1-2,3,4,5,3
  SL_4(2*\delta): 2,0,1-2,3,4,5,1,2,3,0,2,3,4,5,3
2<4<0<5<1<3
  SL_4(\delta): 2,1,0-2,3,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,5,4,3,2,3,5,0,4,2,3,1
2<4<1<5<0<3
  SL_4(\delta): 2,0,1-2,3,5,4,3
  SL_4(2*\delta): 2,0,1-2,3,5,4,3,2,3,5,1,4,2,3,0
2<4<5<0<1<3
  SL_4(\delta): 2,1,0-2,3,5,4,3
  SL_4(2*\delta): 2,1,0-2,3,5,4,3,2,3,0,5,4,2,3,1
2<4<5<1<0<3
  SL_4(\delta): 2,0,1-2,3,5,4,3
  SL_4(2*\delta): 2,0,1-2,3,5,4,3,2,3,1,5,4,2,3,0
2<5<0<4<1<3
  SL_4(\delta): 2,1,0-2,3,4,5,3
  SL_4(2*\delta): 2,1,0-2,3,4,5,3,2,3,4,0,5,2,3,1
2<5<1<4<0<3
  SL_4(\delta): 2,0,1-2,3,4,5,3
  SL_4(2*\delta): 2,0,1-2,3,4,5,3,2,3,4,1,5,2,3,0
2<5<4<0<1<3
  SL_4(\delta): 2,1,0-2,3,4,5,3
  SL_4(2*\delta)

In [ ]:
for i in Dperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
checkHDeltasRepeat(Dperms)

[]

In [ ]:
for i in generateEquivalence(Dperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<2<3<4<0<5 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<4<0<5
1<2<3<5<0<4 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<5<0<4
1<2<4<3<5<0 = 1<2<4<3<0<5
1<2<5<3<4<0 = 1<2<5<3<0<4
1<3<2<4<0<5 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<4<0<5
1<3<2<5<0<4 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<5<0<4
1<3<4<2<5<0 = 1<3<4<2<0<5
1<3<5<2<4<0 = 1<3<5<2<0<4
1<4<2<3<5<0 = 1<4<2<3<0<5
1<4<3<2<5<0 = 1<4<3<2<0<5
1<5<2<3<4<0 

## Type F

In [50]:
Fperms = genPermutations('F',4,4)

  0%|          | 0/120 [00:00<?, ?it/s]

In [51]:
checkMonotoneIncreasingImpDeltaStart(Fperms)

[]

In [52]:
checkDeltaStartImpMonotoneIncreasing(Fperms)

[('0<1<2<3<4', '2'),
 ('0<1<2<3<4', '3'),
 ('0<1<2<3<4', '4'),
 ('0<1<2<3<4', '1'),
 ('0<1<2<3<4', '3,4'),
 ('0<1<2<3<4', '2,3'),
 ('0<1<2<3<4', '1,2'),
 ('0<1<2<3<4', '2,3,4'),
 ('0<1<2<3<4', '2,3,3'),
 ('0<1<2<3<4', '1,2,3'),
 ('0<1<2<3<4', '2,3,4,3'),
 ('0<1<2<3<4', '1,2,3,4'),
 ('0<1<2<3<4', '1,2,3,3'),
 ('0<1<2<3<4', '2,3,4,3,4'),
 ('0<1<2<3<4', '1,2,3,4,3'),
 ('0<1<2<3<4', '1,2,3,3,2'),
 ('0<1<2<3<4', '1,2,3,4,3,2'),
 ('0<1<2<3<4', '1,2,3,4,3,4'),
 ('0<1<2<3<4', '1,2,3,4,3,2,3'),
 ('0<1<2<3<4', '1,2,3,4,3,4,2'),
 ('0<1<2<3<4', '1,2,3,4,3,4,2,3'),
 ('0<1<2<3<4', '1,2,3,4,3,4,2,3,3'),
 ('0<1<2<3<4', '1,2,3,4,3,4,2,3,3,2'),
 ('0<1<2<3<4', '1,2,3,4,3,1,2,3,4,3,2'),
 ('0<1<2<4<3', '2'),
 ('0<1<2<4<3', '3'),
 ('0<1<2<4<3', '4'),
 ('0<1<2<4<3', '1'),
 ('0<1<2<4<3', '4,3'),
 ('0<1<2<4<3', '2,3'),
 ('0<1<2<4<3', '1,2'),
 ('0<1<2<4<3', '2,3,4'),
 ('0<1<2<4<3', '2,3,3'),
 ('0<1<2<4<3', '1,2,3'),
 ('0<1<2<4<3', '2,3,3,4'),
 ('0<1<2<4<3', '1,2,3,4'),
 ('0<1<2<4<3', '1,2,3,3'),
 ('0<1<2<4<3', 

In [53]:
checkDeltaPattern(Fperms,2,printDelta=True)

1<0<3<4<2
  SL_3(\delta): 1,2,3,3,2,0-1,2,3,4,3,4
  SL_3(2*\delta): 1,2,3,3,2,0-1,2,3,4,3,0,1,2,3,4,3,2,1,2,3,4,3,4
2<0<1<4<3
  SL_1(\delta): 2,3,4,1-2,3,4,1,0,2,3,3
  SL_1(2*\delta): 2,3,4,1,2,3,4,1,0-2,3,4,1,2,3,3,2,3,4,1,0,2,3,3
2<0<4<1<3
  SL_1(\delta): 2,3,1,4-2,3,1,4,0,2,3,3
  SL_1(2*\delta): 2,3,1,4,2,3,1,4,0-2,3,1,4,2,3,3,2,3,1,4,0,2,3,3
2<1<0<4<3
  SL_1(\delta): 2,3,4,1-2,3,4,1,0,2,3,3
  SL_1(2*\delta): 2,3,4,1,2,3,4,1,0-2,3,4,1,2,3,3,2,3,4,1,0,2,3,3
2<1<4<0<3
  SL_1(\delta): 2,3,4,1-2,3,4,1,0,2,3,3
  SL_1(2*\delta): 2,3,4,1,2,3,4,1,0-2,3,4,1,2,3,3,2,3,4,1,0,2,3,3
2<1<4<3<0
  SL_1(\delta): 2,3,4,1-2,3,4,1,0,2,3,3
  SL_1(2*\delta): 2,3,4,1,2,3,4,1,0-2,3,4,1,2,3,3,2,3,4,1,0,2,3,3
2<4<0<1<3
  SL_2(\delta): 2,3,1,4-2,3,1,0,4,2,3,3
  SL_2(2*\delta): 2,3,1,4,2,3,1,0,4-2,3,1,4,2,3,3,2,3,1,0,4,2,3,3
2<4<1<0<3
  SL_2(\delta): 2,3,1,4-2,3,1,0,4,2,3,3
  SL_2(2*\delta): 2,3,1,4,2,3,1,0,4-2,3,1,4,2,3,3,2,3,1,0,4,2,3,3
2<4<1<3<0
  SL_2(\delta): 2,3,1,4-2,3,1,0,4,2,3,3
  SL_2(2*\delta): 2,3,

In [54]:
for i in Fperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
checkHDeltasRepeat(Fperms)

[]

In [ ]:
for i in generateEquivalence(Fperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1 = 0<2<3<1<4
0<3<2<4<1 = 0<3<2<1<4
1<2<3<0<4 = 1<2<0<3<4
1<2<3<4<0 = 1<2<0<3<4
1<2<3<4<0 = 1<2<3<0<4
1<2<4<0<3 = 1<2<0<4<3
1<2<4<3<0 = 1<2<0<4<3
1<2<4<3<0 = 1<2<4<0<3
1<3<2<4<0 = 1<3<2<0<4
1<3<4<0<2 = 1<3<0<4<2
1<4<2<3<0 = 1<4<2<0<3
2<0<3<4<1 = 2<0<3<1<4
2<1<3<0<4 = 2<1<0<3<4
2<1<3<4<0 = 2<1<0<3<4
2<1<3<4<0 = 2<1<3<0<4
2<1<4<0<3 = 2<1<0<4<3
2<1<4<3<0 = 2<1<0<4<3
2<1<4<3<0 = 2<1<4<0<3
2<3<0<4<1 = 2<3<0<1<4
2<3<1<4<0 = 2<3<1<0<4
2<3<4<0<1 = 2<3<0<1<4
2<3<4<0<1 = 2<3<0<4<1
2<3<4<1<0 = 2<3<1<0<4
2<3<4<1<0 = 2<3<1<4<0
2<4<1<3<0 = 2<4<1<0<3
3<0<2<4<1 = 3<0<2<1<4
3<1<2<4<0 = 3<1<2<0<4
3<1<4<0<2 = 3<1<0<4<2
3<2<0<4<1 = 3<2<0<1<4
3<2<1<4<0 = 3<2<1<0<4
3<2<4<0<1 = 3<2<0<1<4
3<2<4<0<1 = 3<2<0<4<1
3<2<4<1<0 = 3<2<1<0<4
3<2<4<1<0 = 3<2<1<4<0
4<1<2<3<0 = 4<1<2<0<3
4<2<1<3<0 = 4<2<1<0<3
